# Getting Data From Twitter

## 1. Get Accounts From accounts.txt

In [1]:
cate = ""
all_acc = {}
acc_list = []
f = open("accounts.txt",'r')
for line in f:
    if line[0] == '-':
        continue
    elif line[0] == '=':
        if len(acc_list) > 0:
            all_acc[cate] = acc_list
        cate = line[1:]
        cate = cate.rstrip()
        cate = cate.lstrip()
        acc_list = []
    else:
        sname = line
        sname = sname.rstrip()
        sname = sname.lstrip()
        acc_list.append(sname)

In [2]:
print('There are ... ')
for k in all_acc.keys():
    print(k + ' ' + str(len(all_acc[k])) + ' accounts')

There are ... 
SingleMale 15 accounts
MarriedMale 19 accounts
SingleFemale 14 accounts
MarriedFemale 18 accounts


## 2. Prepare Twitter API for tweet requests

In [3]:
import configparser
from TwitterAPI import TwitterAPI

In [4]:
def get_twitter(config_file):
    config = configparser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

In [5]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)

In [6]:
def get_tweets(twitter, screen_name):
    params = {}
    params['screen_name'] = screen_name
    params['count'] = 50
    params['trim_user'] = True
    params['exclude_replies'] = True
    params['include_rts'] = False
    return robust_request(twitter,'statuses/user_timeline', params)

In [7]:
twitter = get_twitter('twitter.cfg')

## 3. Request and save all tweets to files

In [8]:
nTweets = 0
cate_f = open('categories.txt','w')
got = False
for k in all_acc.keys():
    cate_f.write(k+'\n')
    f = open(k+'.txt','w')
    acc_list = all_acc[k]
    nTweets = 0
    for sname in acc_list:
        req = get_tweets(twitter, sname)
        for r in req:
            tweet = r['text']
            tweet = tweet.replace('\n',' ')
            f.write(tweet+'\n')
            nTweets+=1
    print(str(nTweets)+' tweets have been saved to '+k+'.txt')

cate_f.close()

323 tweets have been saved to SingleMale.txt
483 tweets have been saved to MarriedMale.txt
420 tweets have been saved to SingleFemale.txt
502 tweets have been saved to MarriedFemale.txt
